# Labolatorium 4
## Wyszukiwarka

---
### 1. Przygotowanie zbioru (> 1000 elementów) dokumentów tekstowych w języku angielskim.

Dokumenty znajdują się w folderze `documents`

---
### 2. & 3. Określenie słów kluczowych (termów) potrzebnych do wyznaczenia *bag-of-words* dla każdego dokumentów oraz wyznaczenie *bag-of-words* dla każdego dokumentu $d_j$

W naszym przypadku bazą termów będzie unia wszystkich słów występujących we wszystkich tekstach. Postaramy się pominąć wszelkiego rodzaju znaki interpunkcyjne oraz często występujące słowa w języku angielskim, które nie mają większego znaczenia (np. 'a', 'this', 'of').

Przydatne importy:

In [1]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import os
import string
import scipy.sparse as sparse

try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download('stopwords')

#### zapisywanie i odczyt najważniejszych struktur z pliku

In [2]:
def save_counter_to_file(file_name, bag_of_words):
    text = ""
    for word, count in bag_of_words.items():
        text = text + word + " " + str(count) + "\n"
    filepath = os.path.join(os.getcwd(), file_name)
    with open(filepath, "w", encoding="utf-8") as article_file:
        article_file.write(text)

        
def load_counter_from_file(file_name):
    file_path = os.path.join(os.getcwd(), file_name)

    bag_of_words = Counter()

    
    with open(file_path, "r", encoding="utf8") as text:
            for line in text:
                elems = line.split()  # elems[0] - word, elems[1] - word count
                bag_of_words[elems[0]] = int(elems[1])
                
    return bag_of_words

#### Funkcja, która na podstawie tekstu tworzy wektor *bag-of-words*

In [3]:
def get_bag_of_words_from_text(text):
    # make all article lowercase
    text = text.lower()

    # divide into words (it still includes punctuation)
    words = [word for sentence in sent_tokenize(text) for word in word_tokenize(sentence)]

    # remove meaningless English words such as 'the', 'a' etc.
    english_stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in english_stop_words]

    # remove punctuation
    punctuation = set(string.punctuation)
    punctuation.add("...")
    words = [word for word in words if word not in punctuation]

    # stem words - this makes words with the same meaning equal, e.g. responsibility, responsible => respons
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # remove meaningless 1 or 2 chars words
    words = [word for word in words if len(word) > 2]

    # create a bag_of_words based on this article
    bag_of_words = Counter(words)

    return bag_of_words

#### funkcja, która dla każdego artykułu:
- wczytuje jego tekst i na jego podstawie tworzy wektor cech *bag-of-words* ${d_j}$
- Zapisuje ten wektor do odpowiedniego pliku w katalogu `bags`

W końcu na podstawie wszystkich *bag-of-words* tworzy słownik bazowy dla wszystkich dokumentów i zapisuje go do pliku `base_dictionary`

In [4]:
def process_articles(source_directory = "articles"):
    base_dictionary = Counter()
    
    curr_path = os.getcwd()
    source_path = os.path.join(curr_path, source_directory)
    bags_dir = os.path.join(curr_path, "bags")
    if not os.path.exists(bags_dir):
        os.makedirs(bags_dir)
        
    for file_name in os.listdir(source_path):
        file_path = os.path.join(source_path, file_name)

        with open(file_path, "r", encoding="utf8") as text:

            bag_of_words = get_bag_of_words_from_text(text.read())
            base_dictionary += bag_of_words
            save_counter_to_file(f"bags/{file_name}", bag_of_words)
    
    save_counter_to_file("base_dictionary", base_dictionary)

---
### 4. Budowanie rzadkiej macierzy wektorów cech *term-by-document matrix*.
Wektory ułożone są kolumnowo $A_{m \times n} = [d_1|d_2|...|d_n]$ (m jest liczbą termów w słowniku, n jest liczbą dokumentów).

Poniższa funkcja tworzy macierz cech (pierwsza zwracana wartość), ponadtdo zwraca opis osi Y (termy w odpowiedniej kolejności) jako drugą wartość oraz opis osi X (nazwy dokumentów w odpowiedniej kolejności) jako trzecią wartość

In [5]:
def build_term_by_document_matrix(base_ditionary_name = "base_dictionary", bags_dir = "bags"):
    base_dictionary = load_counter_from_file(base_ditionary_name)
    terms_list = list(base_dictionary)
    
    
    bag_names = os.listdir(bags_dir)
    N = len(bag_names)
    M = len(terms_list)
    
    term_by_document_matrix = sparse.lil_matrix((M, N))

    for j, file_name in enumerate(bag_names):
        bag_of_terms = load_counter_from_file(os.path.join(bags_dir, file_name))

        
        for i, term in enumerate(terms_list):
            term_by_document_matrix[i, j] = bag_of_terms[term]
            
    
    return term_by_document_matrix.tocsr(), terms_list, bag_names

---
### 5. Przetworzenie wstępnie otrzymanego zbioru danych przez *inverse document frequency*. 

$ IDF(w) = log(\frac{N}{n_w}) $

gdzie:\
$n_w$ - liczba dokumentów, w których występuje słowo $w$\
$N$ - całkowita liczba dokumentów

In [6]:
def IDF(row):
    N = row.shape[1]
    n_w = row.count_nonzero()
    return np.log(N / n_w)

def tf_to_idf(term_by_document_matrix):
    for i in range(term_by_document_matrix.shape[0]):
        idf = IDF(term_by_document_matrix[i,:])
        term_by_document_matrix[i,:] *= idf
    return term_by_document_matrix

---
### Czas przetestować wyżej napisane funkcj i utworzyć pełnoprawną macierz *term-by-document*

In [7]:
process_articles()
init_matrix, ax_Y, ax_X = build_term_by_document_matrix()
idf_matrix = tf_to_idf(init_matrix)
idf_matrix.todense()

matrix([[1.09861229, 0.        , 0.        ],
        [1.09861229, 0.        , 0.        ],
        [1.09861229, 0.        , 0.        ],
        [1.09861229, 0.        , 0.        ],
        [1.09861229, 0.        , 0.        ],
        [1.09861229, 0.        , 0.        ],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.81093022, 0.81093022],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511],
        [0.        , 0.40546511, 0.40546511]])

---
### 6. Program pozwalający na wprowadzenie zapytania, który następnie przekształci je do reprezentacji wektorowej $q$

#### Program ma zwrócić $k$ dokumentów najbardziej zbliżonych do podanego zapytania $q$.

Należy użyć korelacji między wektorami jako miary podobieństwa:

$cos(\theta_j) = \frac{q^T d_j}{||q||\cdot||d_j||} = \frac{q^T Ae_j}{||q||\cdot||Ae_j||}$

In [8]:
def find_simillar_documents_beta(query, term_by_document_matrix, terms, documents, k = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))
    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * IDF(term_by_document_matrix[i,:])
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    
    if q_norm == 0:
        return []
    
    similarities = []
    
    for j in range(M):
        d_j = term_by_document_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        similarities.append(((q_T * d_j)[0,0] / (q_norm * d_j_norm), documents[j]))

    return sorted(similarities, key = lambda x: x[0], reverse = True)[:k]

#### Po uruchomieniu poniższej komórki, pojawi się pod nią pole tekstowe. Należy w nie wpisać zapytanie q (sekwencja słów).

In [9]:
query = input("Podaj zapytanie: ")

In [10]:
similar_documents = find_simillar_documents_beta(query, idf_matrix, ax_Y, ax_X)

for _, document in similar_documents:
    print(document)

---
### 7. Zastosowanie normalizacji wektorów $d_j$ i wektora $q$

In [11]:
def find_simillar_documents(query, term_by_document_matrix, terms, documents, k = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))

    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * IDF(term_by_document_matrix[i,:])

    # normalize d_j
    for i in range(M):
        norm = sparse.linalg.norm(term_by_document_matrix[:,i])
        term_by_document_matrix[:,i] /= norm
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    # normalize q
    q_T /= q_norm
    
    if q_norm == 0:
        return []
    
    similarities = q_T * term_by_document_matrix
    result = []
    
    for i in range(M):
        result.append((similarities[0,i], documents[i]))

    return sorted(result, key = lambda x: x[0], reverse = True)[:k]

In [12]:
similar_documents = find_simillar_documents(query, idf_matrix, ax_Y, ax_X)

for _, document in similar_documents:
    print(document)

---
### 8. W celu usunięcia szumu z macierzy A zastosujemy SVD i *low rank approximation*

In [14]:
def find_simillar_documents(query, term_by_document_matrix, terms, documents, limit = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))
    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * IDF(term_by_document_matrix[i,:])
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    
    if q_norm == 0:
        return []
    
    similarities = []
    
    for j in range(M):
        d_j = term_by_document_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        similarities.append(((q_T * d_j)[0,0] / (q_norm * d_j_norm), documents[j]))
    
    for i in range(M):
        result.append((similarities[0,i], documents[i]))

    return sorted(similarities, key = lambda x: x[0], reverse = True)[:limit]